In [1]:
import pandas as pd


from consent.consistency.util import get_scan_dirs, get_scan_root_dir
from consent.util.default_path import get_data_dir
from ooutil.df_util import read_data_files

location = 'ie'
SCAN_DIRS = get_scan_dirs(location)
SCAN_ROOT_DIR = get_scan_root_dir(location)
suffix = '_' + '0k_20k'

In [2]:
cookie_prefs_files = sorted(list(SCAN_ROOT_DIR.glob("cookie_prefs_*.parquet"))) # SCAN_ROOT_DIR/ f'cookie_prefs{suffix}.parquet'
print(f"Cookie prefs files: {[file.name for file in cookie_prefs_files]}")
cookie_prefs = pd.read_parquet(cookie_prefs_files) # pd.read_parquet(cookie_prefs_file)
cookie_prefs.head()

Cookie prefs files: ['cookie_prefs_0k_20k.parquet']


,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name,consent
0,__we_bucket_id,www.wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True
1,s_xxxx,wework.com,730 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True
2,ajs_anonymous_id,wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True
3,OptanonConsent,www.wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True
4,__we_request_id,www.wework.com,Session,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json,True


In [3]:
# TODO: add some flags, this is from pref_category_notebook.ipynb
data_dir = SCAN_ROOT_DIR # get_data_dir('2022-05-30') # '2022-01-14')
cookie_decls_files = sorted(list(SCAN_ROOT_DIR.glob("cookie_decls_*.parquet"))) # data_dir / f'cookie_decls{suffix}.parquet'
print(f"Cookie declaration files: {[file.name for file in cookie_prefs_files]}")
raw_cookie_decls = pd.read_parquet(cookie_decls_files)
# raw_cookie_decls = read_cookie_decls_in_scans(SCAN_DIRS)
print(f"Num cookie declarations: {len(raw_cookie_decls):,d}")
raw_cookie_decls.head()

Cookie declaration files: ['cookie_prefs_0k_20k.parquet']
Num cookie declarations: 282,716


,name,domain,duration,category_id,category,consent_mode,site,lib_name,pattern_name
index,,,,,,,,,
0,__we_bucket_id,www.wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json
1,s_xxxx,wework.com,730 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json
2,ajs_anonymous_id,wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json
3,OptanonConsent,www.wework.com,365 days,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json
4,__we_request_id,www.wework.com,Session,C0001,Strictly Necessary Cookies,always active,wework.com,onetrust,en.json


In [4]:
# Need to read this to report only the sites that have been analyzed
# all_complies_file = SCAN_ROOT_DIR / f'all_complies{suffix}.parquet'
# all_complies = pd.read_parquet(all_complies_file)
# all_complies.head()
all_complies_files = list(SCAN_ROOT_DIR.glob('all_complies_*.parquet'))
print(sorted(f.name for f in all_complies_files))
all_complies = pd.read_parquet(all_complies_files)
all_complies.head()

['all_complies_0k_20k.parquet']


,name,domain,path,site,comply
0,ajs_anonymous_id,.wework.com,/,wework.com,comply
1,sa-user-id-v2,www.wework.com,/,wework.com,omit
2,ajs_user_id,.wework.com,/,wework.com,comply
3,OptanonAlertBoxClosed,.www.wework.com,/,wework.com,comply
4,user,www.wework.com,/,wework.com,comply


In [5]:
site_to_lib = {}
for row in raw_cookie_decls[['site', 'lib_name']].drop_duplicates().itertuples():
    site_to_lib[row.site] = row.lib_name
all_complies['lib_name'] = all_complies['site'].map(site_to_lib)

In [6]:
site_libs = all_complies[['site', 'lib_name']].drop_duplicates()

In [7]:
lib_counts = site_libs['lib_name'].value_counts().to_dict()
lib_counts

{'onetrust': 1341, 'cookiebot': 109, 'termly': 8}

In [8]:
site_to_libname = {row['site']: row['lib_name'] for row in cookie_prefs[['site', 'lib_name']].drop_duplicates().to_dict('records')}
all_complies['lib_name'] = all_complies.site.map(site_to_libname)
all_complies_libs = all_complies.drop_duplicates(subset='site').lib_name.value_counts()
print(all_complies_libs.sum())
# TODO: why this difference??
lib_counts = all_complies_libs
all_complies_libs

1458


onetrust     1340
cookiebot     110
termly          8
Name: lib_name, dtype: int64

In [9]:
inconsist = all_complies[all_complies.comply == 'incorrect'][['site', 'lib_name']].drop_duplicates()
inconsist.head()

,site,lib_name
14,savvis.net,onetrust
35,cengage.com,onetrust
41,oclc.org,onetrust
62,qualtrics.com,onetrust
101,superpages.com,onetrust


In [10]:
inconsist_libs = inconsist['lib_name'].value_counts().to_frame().reset_index().rename(columns={'lib_name': 'count', 'index': 'lib_name'})
inconsist_libs['normalized'] = inconsist_libs.apply(lambda row: row['count'] / lib_counts[row['lib_name']], axis=1)
inconsist_libs['count_percent'] = inconsist_libs.apply(lambda row: f"{row['count']:,d} ({row['normalized']*100:.2f}%)", axis=1)
print("Inconsistent libraries (normalized by number of library instances)")
latex_df = inconsist_libs[['lib_name', 'count_percent']].copy()
latex_df.rename(columns={'lib_name': 'CMP', 'count_percent': '# Violations'}, inplace=True)
latex_df.replace({'onetrust': '\onetrust', 'cookiebot': '\cookiebot', 'termly': r'\termly'}, inplace=True)
print(latex_df.style.hide(axis='index').format_index(escape='latex', axis=1).format(escape='latex').to_latex(hrules=True))

Inconsistent libraries (normalized by number of library instances)


\begin{tabular}{ll}
\toprule
CMP & \# Violations \\
\midrule
\textbackslash onetrust & 1,114 (83.13\%) \\
\textbackslash cookiebot & 82 (74.55\%) \\
\textbackslash termly & 8 (100.00\%) \\
\bottomrule
\end{tabular}

